## Agent with Memory

In [26]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

### **Simple Agent**

The simplest way to use the API is to call the `chat.completions.create` method with a model and a message. We use the previous [00-basic-llm-calls-and-agent.ipynb](./00-basic-llm-calls-and-agent.ipynb) created `Agent` class to illustrate how to use memory in the agent.

In [27]:
class Agent:
    def __init__(
        self,
        name: str = "Agent",
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.agent = OpenAI()

    def invoke(self, message: str) -> str:
        system_prompt = f"You are a {self.role} that {self.instructions}"
        user_question = message
        response = self.agent.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_question},
            ],
            temperature=self.temperature,
        )
        return response.choices[0].message.content

In [28]:
user_message = "What have I asked?"
agent = Agent()

In [29]:
ai_response = agent.invoke(user_message)
print(ai_response)

It seems you haven't asked a specific question yet. How can I assist you today?


### **Memory as a List**

We can use a list to store the memory of the conversation. This is a simple way to store the memory of the conversation. And we can use the `append` method to add a new message to the list.

In [31]:
class Agent:
    def __init__(
        self,
        name: str = "Agent",
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.agent = OpenAI()
        self.memory = [
            {"role": "system", "content": instructions}
        ] # Update: memory as list

    def invoke(self, message: str) -> str:
        self.memory.append({"role": "user", "content": message}) # track user input
        response = self.agent.chat.completions.create(
            model=self.model,
            messages=self.memory,
            temperature=self.temperature,
        )

        ai_response = response.choices[0].message.content
        self.memory.append(
            {"role": "assistant", "content": ai_response}
        ) # track ai response

        return ai_response

In [32]:
agent = Agent()

ai_response = agent.invoke("What's an API")
print(ai_response)

An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.

APIs can be used for various purposes, such as:

1. **Web APIs**: Allow web applications to interact with external services over the internet (e.g., retrieving data from a database, accessing social media platforms).
2. **Library APIs**: Provide a set of functions and procedures for developers to use in their applications (e.g., a graphics library).
3. **Operating System APIs**: Allow applications to interact with the operating system (e.g., file management, network communication).

APIs are essential for building modern software applications, enabling integration and functiona

In [33]:
# check current memory
agent.memory

[{'role': 'system', 'content': 'Help users with any question'},
 {'role': 'user', 'content': "What's an API"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nAPIs can be used for various purposes, such as:\n\n1. **Web APIs**: Allow web applications to interact with external services over the internet (e.g., retrieving data from a database, accessing social media platforms).\n2. **Library APIs**: Provide a set of functions and procedures for developers to use in their applications (e.g., a graphics library).\n3. **Operating System APIs**: Allow applications to interact with the operatin

In [34]:
# Check the model if it can remember the previous message
ai_response = agent.invoke("What have I asked?")
print(ai_response)

You asked, "What's an API?" and I provided an explanation of what an API (Application Programming Interface) is, including its purpose and various types. If you have any more questions or need further clarification, feel free to ask!


### **Creating a more robust memory**

Above method works just fine, but we can create a class to store the memory of the conversation. This is a more robust way. We can further use the `add_message` method to add a new message to the list, and use the `get_messages` method to get the messages from the list.

In [41]:
from typing import List, Dict, Literal

In [42]:
class Memory:
    def __init__(self):
        self.messages: List[Dict[str, str]] = []
    
    def add_message(self, role: Literal['user', 'system', 'assistant'], content: str):
        self.messages.append({
            "role": role,
            "content": content
        })

    def get_messages(self) -> List[Dict[str, str]]:
        return self.messages

We can plug the `Memory` class to the `Agent` class.

In [44]:
class Agent:
    def __init__(
        self,
        name: str = "Agent",
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.agent = OpenAI()
        
        self.memory = Memory()
        self.memory.add_message(
            role="system", 
            content=instructions
        ) # Update: memory as class

    def invoke(self, message: str) -> str:
        self.memory.add_message(role="user", content=message) # track user input
        response = self.agent.chat.completions.create(
            model=self.model,
            messages=self.memory.get_messages(), # query messages from memory
            temperature=self.temperature,
        )

        ai_response = response.choices[0].message.content
        self.memory.add_message(role="assistant", content=ai_response) # track ai response

        return ai_response

Now, let's 's chat with memory and track the memory

In [49]:
agent = Agent()

ai_response = agent.invoke("what's the capital of Brazil")
print(ai_response)

The capital of Brazil is Brasília. It was officially inaugurated as the capital in 1960, designed by the architect Oscar Niemeyer and urban planner Lúcio Costa.


In [50]:
ai_response = agent.invoke("what have I asked?")
print(ai_response)

You asked about the capital of Brazil.


As you can see, the model remembers the previous questions and the memory is not empty.

In [51]:
agent.memory.get_messages()

[{'role': 'system', 'content': 'Help users with any question'},
 {'role': 'user', 'content': "what's the capital of Brazil"},
 {'role': 'assistant',
  'content': 'The capital of Brazil is Brasília. It was officially inaugurated as the capital in 1960, designed by the architect Oscar Niemeyer and urban planner Lúcio Costa.'},
 {'role': 'user', 'content': 'what have I asked?'},
 {'role': 'assistant', 'content': 'You asked about the capital of Brazil.'}]

*Reminder: This notebook demostrate a simple way how to manage memory, but you can use a more robust way to manage memory, like database.*